In [ ]:
toy_dataset = {
    "id": "door-full-new",
    "train-train": [
        "8877",
        "8893",
        "8897",
        "8903",
        "8919",
        "8930",
        "8961",
        "8997",
        "9016",
        "9032",
        "9035",
        "9041",
        "9065",
        "9070",
        "9107",
        "9117",
        "9127",
        "9128",
        "9148",
        "9164",
        "9168",
        "9277",
        "9280",
        "9281",
        "9288",
        "9386",
        "9388",
        "9410",
    ],
    "train-test": ["8867", "8983", "8994", "9003", "9263", "9393"],
    "test": ["8867", "8983", "8994", "9003", "9263", "9393"],
}
# special_req = "half-half"  # "fully-closed"
special_req = None

In [ ]:
from open_anything_diffusion.datasets.flow_trajectory import FlowTrajectoryDataModule

datamodule = FlowTrajectoryDataModule(
    root='/home/yishu/datasets/partnet-mobility/',
    batch_size=1,
    num_workers=30,
    n_proc=2,
    seed=42,
    history=True,
    trajectory_len=1,  # Only used when training trajectory model
    special_req=special_req,  # special_req="fully-closed"
    # # TODO: only for toy training!!!!!
    toy_dataset=toy_dataset,
)
train_loader = datamodule.train_dataloader()

In [ ]:
for data in train_loader:
    break

In [ ]:
data_list = data.to_data_list()
datapoint = data_list[0]
print(datapoint)

In [ ]:
datapoint.delta.shape

In [ ]:
datapoint.flow_history.shape

In [ ]:
torch.cosine

In [ ]:
datapoint.delta

## Current step

In [ ]:
import torch
import numpy as np
from flowbot3d.grasping.agents.flowbot3d import FlowNetAnimation
animation = FlowNetAnimation()

animation.add_trace(
    datapoint.history,
    # torch.as_tensor([pcd[mask]]),
    # torch.as_tensor([flow[mask]]),
    torch.as_tensor([datapoint.history.tolist()]),
    torch.as_tensor([datapoint.flow_history.tolist()]),
    "red",
)

animation.add_trace(
    datapoint.pos,
    # torch.as_tensor([pcd[mask]]),
    # torch.as_tensor([flow[mask]]),
    torch.as_tensor([datapoint.pos.tolist()]),
    torch.as_tensor([datapoint.delta[:, 0, :].tolist()]),
    "red",
)
fig = animation.animate()
fig.show()

## History step

In [ ]:
import torch
import numpy as np
from flowbot3d.grasping.agents.flowbot3d import FlowNetAnimation
animation = FlowNetAnimation()
animation.add_trace(
    datapoint.history,
    # torch.as_tensor([pcd[mask]]),
    # torch.as_tensor([flow[mask]]),
    torch.as_tensor([datapoint.history.tolist()]),
    torch.as_tensor([datapoint.flow_history.tolist()]),
    "red",
)
fig = animation.animate()
fig.show()

### Inference

In [ ]:
class InferenceConfig:
    def __init__(self):
        self.batch_size = 1
        self.trajectory_len = 1

inference_config = InferenceConfig()

class ModelConfig:
    def __init__(self):
        self.num_train_timesteps = 100

model_config = ModelConfig()

In [ ]:
from open_anything_diffusion.models.modules.dit_models import DGDiT, DiT, PN2DiT
from open_anything_diffusion.models.modules.history_translator import HistoryTranslator
from open_anything_diffusion.models.flow_diffuser_hisdit import (
    FlowTrajectoryDiffuserInferenceModule_HisDiT,
)
network = {
    'DiT': DiT(
            in_channels=3 + 3 + 128,
            depth=5,
            hidden_size=128,
            num_heads=4,
            learn_sigma=True,
        ).cuda(),
    'History': HistoryTranslator(history_dim=128, history_len=1).cuda()
}

model = FlowTrajectoryDiffuserInferenceModule_HisDiT(
    network, inference_cfg=inference_config, model_cfg=model_config
)

ckpt_path = '/home/yishu/open_anything_diffusion/logs/train_trajectory_diffuser_hisdit/2024-04-30/07-04-27/checkpoints/epoch=39-step=3520-val_loss=0.00-weights-only.ckpt'
model.load_from_ckpt(ckpt_path)
model.eval()

In [ ]:
pred_flow = model.cuda().predict_step(data.cuda(), 0)[:, 0, :]

In [ ]:
data

In [ ]:
pred_flow.shape

In [ ]:
data.delta[:, 0, :]

In [ ]:
data.flow_history

In [ ]:
pred_flow

In [ ]:
import torch
import numpy as np
from flowbot3d.grasping.agents.flowbot3d import FlowNetAnimation
animation = FlowNetAnimation()
animation.add_trace(
    data.pos.cpu(),
    # torch.as_tensor([pcd[mask]]),
    # torch.as_tensor([flow[mask]]),
    torch.as_tensor([data.pos.cpu().tolist()]),
    torch.as_tensor([pred_flow.cpu().tolist()]),
    "red",
)
fig = animation.animate()
fig.show()